In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the path to your directory
directory = r'C:\Users\nandu\data\train'

# Get a list of all the file paths
image_paths = [os.path.join(directory, img) for img in os.listdir(directory)]

# Load the images
images = [img_to_array(load_img(img_path,target_size=(256, 256))) for img_path in image_paths]
print(images[0])
# Now `images` is a list of numpy arrays representing your images


[[[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [152. 152. 152.]
  [152. 152. 152.]
  [153. 153. 153.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [152. 152. 152.]
  [152. 152. 152.]
  [153. 153. 153.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [152. 152. 152.]
  [152. 152. 152.]
  [153. 153. 153.]]

 ...

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [138. 138. 138.]
  [138. 138. 138.]
  [140. 140. 140.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [138. 138. 138.]
  [138. 138. 138.]
  [140. 140. 140.]]

 [[  0.   0.   0.]
  [  0.   0.   0.]
  [  0.   0.   0.]
  ...
  [139. 139. 139.]
  [140. 140. 140.]
  [141. 141. 141.]]]
